In [1]:
import pandas as pd
import numpy as np
import os
import glob
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.neural_network import MLPClassifier
from sklearn.svm import SVC, LinearSVC
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import SGDClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix, precision_score, recall_score
import seaborn as sns
import matplotlib.pyplot as plt
from scipy.stats import ttest_rel, wilcoxon, shapiro
from sklearn.model_selection import GridSearchCV
import gc
import itertools
from sklearn.utils import resample
import ast
import json
import re

import utils 
import model_train
from constants import *


In [2]:
# List of file paths
df = pd.read_csv(path)
df

,PatientIdentifier,SepsisFlag,GenderDescription,RaceDescription,EthnicGroupDescription,AgeCategory,LOSDays,NumberofVisits,HX_AIDS,HX_ALCOHOL,...,Antiviral_AdminFlag,0-6HoursToFirstAntiviralAdmin,6-12HoursToFirstAntiviralAdmin,12-24HoursToFirstAntiviralAdmin,>24HoursToFirstAntiviralAdmin,Antifungal_AdminFlag,0-6HoursToFirstAntifungalAdmin,6-12HoursToFirstAntifungalAdmin,12-24HoursToFirstAntifungalAdmin,>24HoursToFirstAntifungalAdmin
0,4,1,Male,White,Non-Hispanic or Latino,83,24,3,0,0,...,0,0,0,0,0,1,0,0,0,1
1,19,0,Female,Black or African American,Non-Hispanic or Latino,47,1,2,0,0,...,0,0,0,0,0,0,0,0,0,0
2,23,0,Male,White,Non-Hispanic or Latino,56,3,1,2,2,...,0,0,0,0,0,0,0,0,0,0
3,26,0,Male,Black or African American,Non-Hispanic or Latino,48,1,2,2,2,...,0,0,0,0,0,0,0,0,0,0
4,30,0,Male,Black or African American,Non-Hispanic or Latino,58,4,16,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23101,119770,1,Male,Black or African American,Non-Hispanic or Latino,37,40,2,0,0,...,0,0,0,0,0,1,0,0,0,1
23102,119787,1,Male,Black or African American,Non-Hispanic or Latino,54,3,5,0,0,...,0,0,0,0,0,0,0,0,0,0
23103,119790,0,Male,Unavailable,Unknown,58,1,1,2,2,...,0,0,0,0,0,0,0,0,0,0
23104,119873,0,Male,Black or African American,Non-Hispanic or Latino,73,1,2,2,2,...,0,0,0,0,0,0,0,0,0,0


In [3]:
df_encoded = utils.encode_categorical_from_file(df)
df_encoded

,PatientIdentifier,SepsisFlag,AgeCategory,LOSDays,NumberofVisits,HX_AIDS,HX_ALCOHOL,HX_ANEMDEF,HX_ARTH,HX_BLDLOSS,...,RaceDescription_Unavailable,RaceDescription_White,EthnicGroupDescription_Hispanic or Latino,EthnicGroupDescription_Non-Hispanic or Latino,EthnicGroupDescription_Unknown,FirstLocationTypeCodeAfterArrival_ICU,FirstLocationTypeCodeAfterArrival_NURSE,FirstLocationTypeCodeAfterArrival_OR,FirstLocationTypeCodeAfterArrival_OUTP,FirstLocationTypeCodeAfterArrival_STEPDN
0,4,1,83,24,3,0,0,0,0,0,...,False,True,False,True,False,False,False,True,False,False
1,19,0,47,1,2,0,0,1,0,0,...,False,False,False,True,False,False,False,True,False,False
2,23,0,56,3,1,2,2,2,2,2,...,False,True,False,True,False,False,False,False,False,False
3,26,0,48,1,2,2,2,2,2,2,...,False,False,False,True,False,False,False,False,False,False
4,30,0,58,4,16,0,0,1,0,0,...,False,False,False,True,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23101,119770,1,37,40,2,0,0,0,0,0,...,False,False,False,True,False,False,False,False,False,False
23102,119787,1,54,3,5,0,0,1,0,0,...,False,False,False,True,False,False,True,False,False,False
23103,119790,0,58,1,1,2,2,2,2,2,...,True,False,False,False,True,False,False,False,False,False
23104,119873,0,73,1,2,2,2,2,2,2,...,False,False,False,True,False,False,False,False,False,False


In [4]:
# Define a list of ML models
models = [
    ("DT", DecisionTreeClassifier(criterion='entropy', random_state=42)),
    ("LR", LogisticRegression(solver='lbfgs', max_iter=500, random_state=42, n_jobs=-1)),
    ("NB", GaussianNB()),
    ("NN", MLPClassifier(random_state=42)),
    ("RF", RandomForestClassifier(criterion='entropy', random_state=42)),
    ("SVM", LinearSVC(random_state=42)) 
]

In [5]:
# Initialize an empty DataFrame to store results
results_df = pd.DataFrame()

In [6]:
for name, model in models:
    print(f"Training model: {name}")

    df_encoded = utils.encode_categorical_from_file(df)

    accuracies, precisions, recalls, f1_scores, aucs, losses, tps, tns, fps, fns = model_train.train_model_bootstrap(df_encoded, name, model, n_bootstrap=100, test_size=0.2)

    # Save the results to a dataframe
    result = pd.DataFrame({
        'Accuracy': accuracies,
        'Precision': precisions,
        'Recall': recalls,
        'F1 Score': f1_scores,
        'AUC': aucs,
        'Loss': losses,
        'TP': tps,
        'TN': tns,
        'FP': fps,
        'FN': fns,
        'ML_Model': name
    })

    results_df = pd.concat([results_df, result], ignore_index=False)

Training model: DT
Training model: LR


/opt/anaconda3/lib/python3.12/site-packages/sklearn/linear_model/_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/opt/anaconda3/lib/python3.12/site-packages/sklearn/linear_model/_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_ite

Training model: NB
Training model: NN


/opt/anaconda3/lib/python3.12/site-packages/sklearn/neural_network/_multilayer_perceptron.py:690: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
/opt/anaconda3/lib/python3.12/site-packages/sklearn/neural_network/_multilayer_perceptron.py:690: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
/opt/anaconda3/lib/python3.12/site-packages/sklearn/neural_network/_multilayer_perceptron.py:690: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
/opt/anaconda3/lib/python3.12/site-packages/sklearn/neural_network/_multilayer_perceptron.py:690: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
/opt/anaconda3/lib/python3.12/site-packages/sklearn/neural_network/_

Training model: RF
Training model: SVM


In [7]:
results_df

,Accuracy,Precision,Recall,F1 Score,AUC,Loss,TP,TN,FP,FN,ML_Model
0,0.864777,0.585674,0.558233,0.571624,0.741052,4.873925,417,3580,295,330,DT
1,0.867373,0.583110,0.590231,0.586649,0.754984,4.780646,435,3574,311,302,DT
2,0.858286,0.579728,0.598214,0.588826,0.754761,5.108024,469,3498,340,315,DT
3,0.869321,0.614583,0.605128,0.609819,0.764634,4.702513,472,3546,296,308,DT
4,0.856556,0.569211,0.571429,0.570318,0.742490,5.170260,440,3519,333,330,DT
...,...,...,...,...,...,...,...,...,...,...,...
95,0.903938,0.747460,0.656051,0.698779,0.949712,0.279399,515,3663,174,270,SVM
96,0.892254,0.718350,0.627329,0.669761,0.941724,0.292050,505,3619,198,300,SVM
97,0.898312,0.690658,0.627260,0.657434,0.944700,0.279500,451,3701,202,268,SVM
98,0.895932,0.728198,0.630189,0.675657,0.943870,0.289225,501,3640,187,294,SVM


In [8]:
# Save the results to a CSV file
results_df.to_csv(f"/Users/yusiwei/Library/CloudStorage/OneDrive-Personal/research/Fourth Year Paper/Experiments/2nd experiments/Baseline experiment results/Sepsis_baseline_ML_results.csv", index=False)